Introduction:

This code implements a machine learning pipeline using XGBoost for predicting early dialysis recommendations in medical cases. The pipeline includes data preprocessing, handling imbalanced data using SMOTE, and model training with optimized hyperparameters. The implementation uses XGBoost's native API with DMatrix format and includes early stopping to prevent overfitting. Finally, it provides a practical function predict_early_dialysis() that allows medical professionals to input patient parameters and receive a recommendation about early dialysis.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import xgboost as xgb

# Load the dataset
df = pd.read_csv('cleaned_data.csv')


# Update features as per the requirement
features = ['calcium_max', 'creatinine_min', 'aki_stage', 'aniongap_min', 'calcium_min', 'pt_max']

# Drop rows with missing values in the selected columns
df_clean = df.dropna(subset=features + ['delay_rrt'])

# Splitting data into input features (X) and target (y)
X = df_clean[features]
y = df_clean['delay_rrt']


# Splitting data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into a smaller training set and a validation set
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Applying SMOTE to balance the training set
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_final, y_train_final)

# Convert the datasets into DMatrix format (required by XGBoost's native API)
dtrain = xgb.DMatrix(X_resampled, label=y_resampled)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test)

# Use the best parameters found from GridSearchCV (based on the previous GridSearch results)
best_params = {
    'n_estimators': 100,        # Best n_estimators from GridSearchCV
    'max_depth': 5,             # Best max_depth from GridSearchCV
    'learning_rate': 0.1,       # Best learning rate from GridSearchCV
    'subsample': 0.8,           # Best subsample from GridSearchCV
    'colsample_bytree': 0.8,    # Best colsample_bytree from GridSearchCV
    'scale_pos_weight': 2,      # Best scale_pos_weight from GridSearchCV
    'objective': 'binary:logistic',
    'eval_metric': 'logloss'
}

# Train the XGBoost model with early stopping using the DMatrix API
evals = [(dval, 'validation')]
xgb_model = xgb.train(
    best_params, 
    dtrain, 
    num_boost_round=500,         # Maximum number of boosting rounds
    early_stopping_rounds=10,    # Stop training if no improvement in 10 rounds
    evals=evals, 
    verbose_eval=True
)

# Evaluating the model with the test set
y_pred_balanced = np.round(xgb_model.predict(dtest))
print("Classification Report:\n", classification_report(y_test, y_pred_balanced))

# Function to predict early dialysis based on user input
def predict_early_dialysis(calcium_max, creatinine_min, aki_stage, aniongap_min, calcium_min, pt_max, model, feature_names):
    # Create a numpy array with the input data
    input_data = np.array([[calcium_max, creatinine_min, aki_stage, aniongap_min, calcium_min, pt_max]])
    
    # Create a DMatrix with feature names to match the training data
    dinput = xgb.DMatrix(input_data, feature_names=feature_names)
    
    # Use the trained model to predict the class (0 or 1)
    prediction = np.round(model.predict(dinput))
    
    # Output the result
    if prediction[0] == 1:
        return "Early dialysis is recommended."
    else:
        return "Early dialysis is not required."

# Example of user input and model prediction
feature_names = X_train.columns.tolist()
result = predict_early_dialysis(
    calcium_max=9.2, 
    creatinine_min=0.3, 
    aki_stage=2, 
    aniongap_min=11, 
    calcium_min=7.7, 
    pt_max=14.5, 
    model=xgb_model, 
    feature_names=feature_names
)
print(result)


[0]	validation-logloss:0.54002
[1]	validation-logloss:0.53603
[2]	validation-logloss:0.53249
[3]	validation-logloss:0.52662
[4]	validation-logloss:0.52421
[5]	validation-logloss:0.52040
[6]	validation-logloss:0.51979
[7]	validation-logloss:0.51980
[8]	validation-logloss:0.51778
[9]	validation-logloss:0.51417
[10]	validation-logloss:0.51234
[11]	validation-logloss:0.51168
[12]	validation-logloss:0.51034
[13]	validation-logloss:0.50804
[14]	validation-logloss:0.50658
[15]	validation-logloss:0.50641
[16]	validation-logloss:0.50666
[17]	validation-logloss:0.50288
[18]	validation-logloss:0.50343
[19]	validation-logloss:0.50269
[20]	validation-logloss:0.50497
[21]	validation-logloss:0.50323
[22]	validation-logloss:0.50432
[23]	validation-logloss:0.50711
[24]	validation-logloss:0.50795
[25]	validation-logloss:0.50840
[26]	validation-logloss:0.50790
[27]	validation-logloss:0.50659
[28]	validation-logloss:0.50748
Classification Report:
               precision    recall  f1-score   support

   

C:\Users\Harshvardhan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:158: UserWarning: [07:42:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


Observations:

The XGBoost model demonstrates effective learning with validation loss decreasing from 0.54 to 0.50, showing good convergence over training iterations.

 The model achieves a solid weighted average F1-score of 0.77, indicating reliable performance in predicting early dialysis recommendations. Despite using SMOTE for balance, the difference between macro averages (0.59) and weighted averages (0.77) suggests some remaining class imbalance effects, though the model maintains good precision (0.76) and recall (0.79) for practical clinical applications. 
 
 The early stopping mechanism appears to be working effectively, with optimal performance around iterations 15-17, helping prevent overfitting while maintaining predictive accuracy.